In [2]:
import rasterio
import numpy as np
import cv2
from rasterio.features import shapes
from shapely.geometry import shape, mapping
from scipy import ndimage
import matplotlib.pyplot as plt
from pathlib import Path
import json
from collections import defaultdict

# --- Configuration ---Preprocessing/NewImagesTesting/new_test_images/tile_HK_00110.png
IMAGE_PATH = "new_test_images/tile_HK_00110.png"
mask_path = "outputs2/tile_HK_00110_masks.tif"
clean_mask_path = "masks_clean.tif"
boundaries_path = "masks_boundaries.tif"
polygons_geojson = "polygons.geojson"

# --- CONTAINMENT-AWARE MERGING ---
USE_CONTAINMENT_MERGING = True
SMALL_FRAGMENT_THRESHOLD = 10000      # Lower = merge more small fragments
CONTAINMENT_THRESHOLD = 0.3          # Lower = more aggressive merging (30% overlap)
TOUCH_MERGE_THRESHOLD = 0.5          # Lower = merge small fragments more easily

# --- Morphological Processing ---
USE_MORPHOLOGY = False
MORPH_CLOSE_KERNEL = 3
MORPH_CLOSE_ITERATIONS = 1

# --- Filtering Parameters (VERY PERMISSIVE - KEEP EVERYTHING) ---
MIN_AREA = 100                       # Very small - keep tiny regions
MAX_AREA = 20000000
MIN_COMPACTNESS = 0.001              # Very low - accept any shape
MIN_SOLIDITY = 0.001                 # Very low - accept fragmented shapes
MAX_ASPECT_RATIO = 1000.0            # Very high - accept elongated shapes
MIN_CONVEXITY = 0.001                # Very low - accept concave shapes
MIN_EXTENT = 0.001                   # Very low - accept sparse shapes

BOUNDARY_THICKNESS = 5
POLYGON_LINE_WIDTH = 3               # Thicker polygon lines
CREATE_BOUNDARY_FILE = True
SAVE_GEOJSON = True


def load_sam_mask(mask_path):
    """Load SAM mask efficiently"""
    with rasterio.open(mask_path) as src:
        mask_data = src.read()
        profile = src.profile.copy()
        transform = src.transform
        crs = src.crs
    
    print(f"\n{'='*60}")
    print(f"MASK ANALYSIS")
    print(f"{'='*60}")
    print(f"Shape: {mask_data.shape}, Dtype: {mask_data.dtype}")
    
    if mask_data.ndim == 3 and mask_data.shape[0] > 1:
        print(f"✓ Multi-band mask: {mask_data.shape[0]} bands")
        instance_mask = np.zeros(mask_data.shape[1:], dtype=np.uint32)
        
        for band_idx in range(mask_data.shape[0]):
            band = mask_data[band_idx]
            if np.any(band > 0):
                band_mask = band > 0
                instance_mask[band_mask & (instance_mask == 0)] = band_idx + 1
        
        del mask_data, band, band_mask
    else:
        instance_mask = mask_data.squeeze().astype(np.uint32)
        unique_vals = np.unique(instance_mask)
        
        if len(unique_vals[unique_vals > 0]) <= 1:
            print(f"⚠ Binary mask - labeling connected components")
            instance_mask, _ = ndimage.label(instance_mask > 0)
            instance_mask = instance_mask.astype(np.uint32)
        
        del mask_data
    
    num_instances = len(np.unique(instance_mask)) - 1
    print(f"✓ Loaded {num_instances} objects")
    
    return instance_mask, profile, transform, crs


def apply_containment_merging(instance_mask):
    """ULTRA AGGRESSIVE merging - merge ALL small fragments into nearest neighbor"""
    if not USE_CONTAINMENT_MERGING:
        return instance_mask
    
    print(f"\n{'='*60}")
    print(f"ULTRA AGGRESSIVE MERGING - NO SMALL FRAGMENTS LEFT")
    print(f"{'='*60}")
    
    unique_ids = np.unique(instance_mask)
    unique_ids = unique_ids[unique_ids > 0]
    
    # Calculate sizes
    max_id = unique_ids.max()
    flat_mask = instance_mask.ravel()
    obj_sizes = np.bincount(flat_mask, minlength=max_id + 1)
    del flat_mask
    
    # Classify objects
    small_objs = unique_ids[obj_sizes[unique_ids] < SMALL_FRAGMENT_THRESHOLD]
    large_objs = unique_ids[obj_sizes[unique_ids] >= SMALL_FRAGMENT_THRESHOLD]
    
    print(f"Large objects: {len(large_objs)}, Small fragments: {len(small_objs)}")
    print(f"Strategy: Merge EVERY small fragment into its nearest neighbor")
    
    if len(small_objs) == 0:
        print("✓ No small fragments to merge")
        return instance_mask
    
    # Build merge map - FORCE merge for ALL small fragments
    merge_map = {}
    kernel = np.ones((5, 5), np.uint8)  # LARGER kernel for better neighbor detection
    
    print(f"Processing {len(small_objs)} fragments...")
    
    for idx, small_id in enumerate(small_objs):
        if idx % 50 == 0 and idx > 0:
            print(f"  Merged {idx}/{len(small_objs)}...")
        
        small_mask = (instance_mask == small_id).astype(np.uint8)
        coords = np.argwhere(small_mask > 0)
        if len(coords) == 0:
            continue
        
        y_min, x_min = coords.min(axis=0)
        y_max, x_max = coords.max(axis=0)
        
        # Large padding to find ANY neighbor
        pad = 10
        y_min = max(0, y_min - pad)
        x_min = max(0, x_min - pad)
        y_max = min(instance_mask.shape[0], y_max + pad + 1)
        x_max = min(instance_mask.shape[1], x_max + pad + 1)
        
        small_crop = small_mask[y_min:y_max, x_min:x_max]
        mask_crop = instance_mask[y_min:y_max, x_min:x_max]
        
        # AGGRESSIVE dilation to find neighbors
        dilated = cv2.dilate(small_crop, kernel, iterations=3)
        neighbor_region = (dilated > 0) & (small_crop == 0)
        
        neighbor_ids = np.unique(mask_crop[neighbor_region])
        neighbor_ids = neighbor_ids[(neighbor_ids > 0) & (neighbor_ids != small_id)]
        
        # If NO neighbors found with dilation, expand search further
        if len(neighbor_ids) == 0:
            # Try even BIGGER dilation
            big_kernel = np.ones((11, 11), np.uint8)
            dilated = cv2.dilate(small_crop, big_kernel, iterations=5)
            neighbor_region = (dilated > 0) & (small_crop == 0)
            neighbor_ids = np.unique(mask_crop[neighbor_region])
            neighbor_ids = neighbor_ids[(neighbor_ids > 0) & (neighbor_ids != small_id)]
        
        if len(neighbor_ids) == 0:
            # Still no neighbor? Merge into closest large object
            if len(large_objs) > 0:
                # Find closest large object by distance
                small_coords = np.argwhere(small_mask > 0)
                centroid = small_coords.mean(axis=0)
                
                min_dist = float('inf')
                closest_large = None
                
                for large_id in large_objs:
                    large_coords = np.argwhere(instance_mask == large_id)
                    if len(large_coords) > 0:
                        large_centroid = large_coords.mean(axis=0)
                        dist = np.linalg.norm(centroid - large_centroid)
                        if dist < min_dist:
                            min_dist = dist
                            closest_large = large_id
                
                if closest_large is not None:
                    merge_map[small_id] = closest_large
            continue
        
        # RULE: Merge into ANY neighbor (prefer large, but accept small too)
        # Priority: large objects > bigger small objects
        large_neighbors = [n for n in neighbor_ids if n in large_objs]
        
        if large_neighbors:
            # Merge into largest neighbor
            best_id = max(large_neighbors, key=lambda x: obj_sizes[x])
            merge_map[small_id] = best_id
        else:
            # No large neighbors, merge into biggest small neighbor
            best_id = max(neighbor_ids, key=lambda x: obj_sizes[x])
            merge_map[small_id] = best_id
        
        del small_mask, small_crop, mask_crop, dilated, neighbor_region
    
    # Apply merges
    if not merge_map:
        print("⚠ Warning: No merges created (this shouldn't happen)")
        return instance_mask
    
    print(f"\nApplying {len(merge_map)}/{len(small_objs)} merges...")
    
    # Resolve transitive merges
    def resolve_target(obj_id, visited=None):
        if visited is None:
            visited = set()
        if obj_id in visited or obj_id not in merge_map:
            return obj_id
        visited.add(obj_id)
        return resolve_target(merge_map[obj_id], visited)
    
    # Apply all merges
    for small_id, target_id in merge_map.items():
        final_target = resolve_target(target_id)
        instance_mask[instance_mask == small_id] = final_target
    
    # Renumber sequentially
    unique_after = np.unique(instance_mask)
    unique_after = unique_after[unique_after > 0]
    
    renumber_map = np.zeros(max_id + 1, dtype=np.uint32)
    for new_id, old_id in enumerate(unique_after, start=1):
        renumber_map[old_id] = new_id
    
    instance_mask = renumber_map[instance_mask]
    
    merged_count = len(unique_ids) - len(unique_after)
    print(f"✓ Objects: {len(unique_ids)} → {len(unique_after)}")
    print(f"✓ Merged {merged_count} small fragments ({merged_count/len(unique_ids)*100:.1f}%)")
    
    # Check if any small fragments remain
    remaining_sizes = np.bincount(instance_mask.ravel())[1:]
    remaining_small = np.sum(remaining_sizes < SMALL_FRAGMENT_THRESHOLD)
    if remaining_small > 0:
        print(f"⚠ Warning: {remaining_small} small fragments still remain")
    else:
        print(f"✓ SUCCESS: All small fragments merged!")
    
    return instance_mask


def calculate_shape_metrics(obj_mask):
    """Calculate shape metrics"""
    contours, _ = cv2.findContours(obj_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    contour = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    if area == 0 or perimeter == 0:
        return None
    
    x, y, w, h = cv2.boundingRect(contour)
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull)
    hull_perimeter = cv2.arcLength(hull, True)
    
    return {
        'area': area,
        'compactness': (4 * np.pi * area) / (perimeter ** 2),
        'solidity': area / hull_area if hull_area > 0 else 0,
        'aspect_ratio': max(w, h) / min(w, h) if min(w, h) > 0 else 0,
        'extent': area / (w * h) if (w * h) > 0 else 0,
        'convexity': hull_perimeter / perimeter if perimeter > 0 else 0
    }


def filter_objects_by_quality(instance_mask):
    """Very permissive filtering - keep almost everything"""
    unique_ids = np.unique(instance_mask)
    unique_ids = unique_ids[unique_ids > 0]
    
    print(f"\n{'='*60}")
    print(f"FILTERING {len(unique_ids)} OBJECTS (PERMISSIVE)")
    print(f"{'='*60}")
    
    cleaned_mask = np.zeros_like(instance_mask, dtype=np.uint32)
    new_id = 1
    removed = {'too_small': 0, 'too_large': 0, 'poor_quality': 0}
    
    for inst_id in unique_ids:
        obj_mask = (instance_mask == inst_id).astype(np.uint8)
        metrics = calculate_shape_metrics(obj_mask)
        
        if metrics is None:
            removed['too_small'] += 1
            continue
        
        # Very permissive filters
        if metrics['area'] < MIN_AREA:
            removed['too_small'] += 1
        elif metrics['area'] > MAX_AREA:
            removed['too_large'] += 1
        elif (metrics['compactness'] < MIN_COMPACTNESS or
              metrics['solidity'] < MIN_SOLIDITY or
              metrics['aspect_ratio'] > MAX_ASPECT_RATIO or
              metrics['convexity'] < MIN_CONVEXITY or
              metrics['extent'] < MIN_EXTENT):
            removed['poor_quality'] += 1
        else:
            cleaned_mask[obj_mask > 0] = new_id
            new_id += 1
    
    total_kept = new_id - 1
    total_removed = sum(removed.values())
    
    print(f"✓ Kept: {total_kept}, Removed: {total_removed}")
    for reason, count in removed.items():
        if count > 0:
            print(f"  • {reason}: {count}")
    
    return cleaned_mask


def extract_polygons_per_object(instance_mask, transform):
    """Extract polygons"""
    unique_ids = np.unique(instance_mask)
    unique_ids = unique_ids[unique_ids > 0]
    
    print(f"\n{'='*60}")
    print(f"EXTRACTING {len(unique_ids)} POLYGONS")
    print(f"{'='*60}")
    
    polygons = []
    
    for inst_id in unique_ids:
        obj_mask = (instance_mask == inst_id).astype(np.uint8)
        area = np.count_nonzero(obj_mask)
        
        for geom, val in shapes(obj_mask, mask=obj_mask, transform=transform):
            if val > 0:
                polygons.append({
                    'polygon': shape(geom),
                    'id': int(inst_id),
                    'area': int(area)
                })
                break
    
    print(f"✓ Extracted {len(polygons)} polygons")
    return polygons


def save_geojson(polygons, output_path, crs):
    """Save polygons to GeoJSON"""
    features = [{
        "type": "Feature",
        "properties": {"id": p['id'], "area_pixels": p['area']},
        "geometry": mapping(p['polygon'])
    } for p in polygons]
    
    geojson = {
        "type": "FeatureCollection",
        "crs": {"type": "name", "properties": {"name": str(crs) if crs else "EPSG:4326"}},
        "features": features
    }
    
    with open(output_path, 'w') as f:
        json.dump(geojson, f, indent=2)
    
    print(f"✓ GeoJSON saved: {output_path} ({len(features)} polygons)")


def extract_boundaries(instance_mask, thickness=4):
    """Extract boundaries"""
    boundaries = np.zeros_like(instance_mask, dtype=np.uint8)
    kernel = np.ones((3, 3), np.uint8)
    
    unique_ids = np.unique(instance_mask)
    unique_ids = unique_ids[unique_ids > 0]
    
    for inst_id in unique_ids:
        obj_mask = (instance_mask == inst_id).astype(np.uint8)
        edges = cv2.morphologyEx(obj_mask, cv2.MORPH_GRADIENT, kernel)
        boundaries = np.maximum(boundaries, edges)
    
    if thickness > 1:
        thick_kernel = np.ones((thickness, thickness), np.uint8)
        boundaries = cv2.dilate(boundaries, thick_kernel, iterations=1)
    
    return boundaries


def load_real_image(image_path):
    """Load and normalize image"""
    with rasterio.open(image_path) as src:
        img = src.read()
    
    if img.shape[0] == 3:
        img = np.transpose(img, (1, 2, 0))
    else:
        img = img.squeeze()
    
    img = img.astype(np.float32)
    img = (img - img.min()) / (img.max() - img.min() + 1e-6)
    return img


def visualize_results(instance_mask, boundaries, polygons, save_path, real_image=None):
    """Create visualization with THICK polygon lines - DIRECTLY FROM MASK"""
    num_objects = len(np.unique(instance_mask)) - 1
    
    fig, axs = plt.subplots(2, 2, figsize=(20, 20))
    
    if real_image is not None:
        # Original image
        axs[0, 0].imshow(real_image)
        axs[0, 0].set_title("Original Image", fontsize=16, fontweight='bold')
        axs[0, 0].axis("off")
        
        # Image + Boundaries
        overlay = real_image.copy()
        if overlay.ndim == 2:
            overlay = np.stack([overlay]*3, axis=-1)
        overlay[boundaries > 0] = [1.0, 0.0, 0.0]
        
        axs[0, 1].imshow(overlay)
        axs[0, 1].set_title("Image + Boundaries", fontsize=16, fontweight='bold')
        axs[0, 1].axis("off")
    
    # Instance mask
    axs[1, 0].imshow(instance_mask, cmap="nipy_spectral", interpolation='nearest')
    axs[1, 0].set_title(f"Instance Mask ({num_objects} objects)", fontsize=16, fontweight='bold')
    axs[1, 0].axis("off")
    
    # Image + POLYGONS BOUNDARIES - Draw directly from mask contours
    if real_image is not None:
        axs[1, 1].imshow(real_image)
        
        print(f"\nDrawing {num_objects} object boundaries on visualization...")
        
        # Get unique object IDs
        unique_ids = np.unique(instance_mask)
        unique_ids = unique_ids[unique_ids > 0]
        
        # Draw contour for EACH object
        for idx, obj_id in enumerate(unique_ids):
            # Extract this object's mask
            obj_mask = (instance_mask == obj_id).astype(np.uint8)
            
            # Find contours in PIXEL coordinates
            contours, _ = cv2.findContours(obj_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Draw each contour
            for contour in contours:
                # Contour is in [N, 1, 2] format, reshape to [N, 2]
                contour = contour.squeeze()
                
                if contour.ndim == 2 and len(contour) > 2:
                    # Close the polygon by appending first point
                    contour_closed = np.vstack([contour, contour[0]])
                    
                    # Plot with THICK lines (x, y order for matplotlib)
                    axs[1, 1].plot(contour_closed[:, 0], contour_closed[:, 1], 
                                  color='lime', linewidth=POLYGON_LINE_WIDTH, 
                                  alpha=0.9, solid_capstyle='round', solid_joinstyle='round')
            
            if (idx + 1) % 10 == 0:
                print(f"  Drew {idx + 1}/{num_objects} objects...")
        
        axs[1, 1].set_title(f"Image + {num_objects} Object Boundaries (Thick Lines)", 
                           fontsize=16, fontweight='bold')
        axs[1, 1].axis("off")
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=200, bbox_inches='tight')
    print(f"✓ Visualization saved: {save_path}")
    
    # SHOW in notebook
    plt.show()
    
    return fig


def main(mask_path,image_path):
    """Main pipeline"""
    if not Path(mask_path).exists():
        raise FileNotFoundError(f"Mask file not found: {mask_path}")
    
    print(f"\n{'='*60}")
    print(f"SAM MASK → FILTERED POLYGONS")
    print(f"{'='*60}")
    
    # Load
    instance_mask, profile, transform, crs = load_sam_mask(mask_path)
    
    # Merge (aggressive)
    instance_mask = apply_containment_merging(instance_mask)
    
    # Filter (very permissive)
    instance_mask = filter_objects_by_quality(instance_mask)
    
    # Extract
    polygons = extract_polygons_per_object(instance_mask, transform)
    boundaries = extract_boundaries(instance_mask, BOUNDARY_THICKNESS)
    
    print(f"\n{'='*60}")
    print(f"FINAL POLYGON COUNT: {len(polygons)}")
    print(f"{'='*60}")
    
    # Save
    print(f"\n{'='*60}")
    print(f"SAVING FILES")
    print(f"{'='*60}")
    
    profile.update({'count': 1, 'dtype': 'uint32', 'compress': 'lzw', 'nodata': 0})
    
    with rasterio.open(clean_mask_path, "w", **profile) as dst:
        dst.write(instance_mask, 1)
    print(f"✓ Instance mask: {clean_mask_path}")
    
    if CREATE_BOUNDARY_FILE:
        profile['dtype'] = 'uint8'
        with rasterio.open(boundaries_path, "w", **profile) as dst:
            dst.write(boundaries, 1)
        print(f"✓ Boundaries: {boundaries_path}")
    
    if SAVE_GEOJSON:
        save_geojson(polygons, polygons_geojson, crs)
    
    # Visualize
    print(f"\n{'='*60}")
    print(f"VISUALIZATION")
    print(f"{'='*60}")
    real_image = load_real_image(image_path)
    fig = visualize_results(instance_mask, boundaries, polygons, "polygons_visualization.png", real_image)
    
    print(f"\n{'='*60}")
    print(f"✓ COMPLETE!")
    print(f"{'='*60}")
    print(f"\n📊 Summary:")
    print(f"  • Total polygons: {len(polygons)}")
    print(f"  • GeoJSON features: {len(polygons)}")
    print(f"  • All polygons drawn with {POLYGON_LINE_WIDTH}px thick lines")
    print(f"  • NO small regions removed - everything merged")
    
    return instance_mask, boundaries, polygons, real_image, fig
    
if __name__ == "__main__":
    try:
        instance_mask, boundaries, polygons, real_image, fig = main()
        # instance_mask, boundaries, polygons, real_image, fig = main()
    except Exception as e:
        print(f"\nERROR: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from samgeo import SamGeo2
import cv2

# Initialize SAM
sam2 = SamGeo2(
    model_id="sam2-hiera-large",
    apply_postprocessing=True,
    points_per_side=64,        
    points_per_batch=128,
    pred_iou_thresh=0.7,         
    stability_score_thresh=0.85,
    stability_score_offset=0.7,
    crop_n_layers=1,          
    box_nms_thresh=0.8,
    min_mask_region_area=10.0,   
    use_m2m=True,              
)


def overlay_mask_downsampled(image_path, output_dir="outputs2", max_size=1024, alpha=0.4, overlay_boundaries=True):
    """
    Overlay instance mask on a downsampled original image.
    
    Parameters
    ----------
    image_path : str
        Path to the original image.
    output_dir : str
        Directory to save intermediate outputs.
    max_size : int
        Maximum width or height for visualization.
    alpha : float
        Transparency for overlay.
    overlay_boundaries : bool
        Whether to overlay boundaries in red.
    """
    os.makedirs(output_dir, exist_ok=True)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    mask_path = os.path.join(output_dir, f"{base_name}_masks.tif")
    
    # Generate SAM masks
    sam2.generate(image_path)
    sam2.save_masks(mask_path)
    sam2.show_anns(
    axis="off",
    alpha=0.7)
    # Run your main pipeline to get instance mask
    try:
        instance_mask, boundaries, polygons, real_image, fig = main(mask_path,image_path)
    except Exception as e:
        print(f"ERROR processing {mask_path}: {e}")
        import traceback
        traceback.print_exc()
        return None

    # Open original image
    with rasterio.open(image_path) as src:
        img = src.read([1, 2, 3])  # Assuming RGB
        img = img.transpose(1, 2, 0).astype(float)  # H x W x C
        img = (img - img.min()) / (img.max() - img.min())  # Normalize to 0-1

        orig_h, orig_w = img.shape[:2]
        scale = min(max_size / orig_w, max_size / orig_h, 1.0)

        if scale < 1.0:
            new_w, new_h = int(orig_w * scale), int(orig_h * scale)
            img_ds = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
            mask_ds = cv2.resize(instance_mask.astype(np.float32), (new_w, new_h), interpolation=cv2.INTER_NEAREST)
            boundaries_ds = cv2.resize(boundaries.astype(np.uint8), (new_w, new_h), interpolation=cv2.INTER_NEAREST)
        else:
            img_ds = img
            mask_ds = instance_mask
            boundaries_ds = boundaries

        # Create colored overlay
        num_objects = int(np.max(mask_ds))  # Fix TypeError by casting to int
        if num_objects == 0:
            overlay = img_ds
        else:
            import matplotlib.cm as cm
            colors = cm.nipy_spectral(np.linspace(0, 1, num_objects + 1))[:, :3]  # RGB only
            mask_rgb = colors[mask_ds.astype(int)]
            overlay = (1 - alpha) * img_ds + alpha * mask_rgb

        # Overlay boundaries if requested
        if overlay_boundaries:
            overlay[boundaries_ds > 0] = [1.0, 0, 0]  # Red boundaries

        # Plot and save
        plt.figure(figsize=(12, 12))
        plt.imshow(overlay)
        plt.axis("off")
        plt.title(f"{base_name}: {num_objects} objects")
        save_path = os.path.join(output_dir, f"{base_name}_overlay.png")
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"Overlay saved: {save_path}")

    return {
        "image_path": image_path,
        "mask_path": mask_path,
        "overlay_path": save_path,
        "num_objects": num_objects
    }


# ---- Main loop ----
image_folder = "new_test_images"
results = []

for file in sorted(os.listdir(image_folder)):
    if file.lower().endswith(".png"):
        full_path = os.path.join(image_folder, file)
        print(f"Processing {full_path} ...")
        res = overlay_mask_downsampled(full_path)
        if res:
            results.append(res)

print("All images processed.")


In [ ]:
!nvidia-smi

In [ ]:
!kill -9 2155382

In [ ]:
!kill -9 2461153